In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
from d00_utils.db_utils import dbReadWriteMeasurement
io_measurement = dbReadWriteMeasurement()

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
from d05_measurement.write_groundtruth import write_groundtruth
write_groundtruth()

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
ground_truths_df = io_measurement.get_table('ground_truths')
print(ground_truths_df.shape)
ground_truths_df.head()

(50218, 7)


,ground_truth_id,study_id,instance_id,file_name,measurement_name,measurement_unit,measurement_value
0,0,39531,696112,a_39531_8Z0BWZGH,VTD(MDD-ps4),mL,138.023153580371
1,1,39531,696112,a_39531_8Z0BWZGH,VTS(MDD-ps4),mL,96.1217048828963
2,2,39552,696426,a_39552_8ZGVB5TY,VTD(MDD-ps4),mL,147.6939058118
3,3,39552,696426,a_39552_8ZGVB5TY,VTS(MDD-ps4),mL,65.1024746074442
4,4,39553,696438,a_39553_8ZGVCN6C,VTD(MDD-ps4),mL,122.502085154058
